In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import accuracy_score

from sklearn.linear_model import LogisticRegression

In [ ]:
# Ucitavanje training podataka
train_data = pd.read_csv('./train.csv')
# Ucitvanje test podataka
test_data = pd.read_csv('./test.csv')

In [ ]:
# Kratak pregled training podataka
train_data.info()
# Kratak pregled test podataka
test_data.info()

In [ ]:
# Iz nepoznatog razloga nece da se zameni, pa je odradjena rucna zamena
test_data.loc[test_data['PassengerId'] == 1306, 'Name'] = 'Dona.'

# Spajanje training i test podataka u jednu tabelu
train_test = pd.concat([train_data, test_data], axis=0)
# Zadrzavanje indeksa
train_test = train_test.reset_index(drop=True) 
# Pclass kolona sadrzi numericke podatke kategorickog tipa, pa se pretvara u string kako bi radila funkcija get_dumies
train_test['Pclass'] = train_test['Pclass'].apply(lambda x: str(x))

In [ ]:
# Odbacivanje kolona Ticket i Cabin zato sto ne sadrze korisne informacije
train_test = train_test.drop(['Ticket', 'Cabin'], axis=1)

In [ ]:
# Rucno izdvojene sve titule
honorifics = ['Mr.',
              'Mrs.',
              'Miss.',
              'Rev.',
              'Master.',
              'Dr.',
              'Don.',
              'Sir.',
              'Mlle.',
              'Mme.',
              'Major.',
              'Countess.',
              'Lady.',
              'Dona.',
              'Capt.']

# Funkcija za zamenu imena titulom
def change_to_honorific(name):
    for honorific in honorifics:
        if honorific in name:
            return honorific

# Zamena svih imena odgovarajucom titulom
train_test['Name'] = train_test['Name'].apply(change_to_honorific)

In [ ]:
# Izdvajanje svih kategorija
train_test = pd.get_dummies(train_test)
# Funkcija get_dumies nove kolone popunjava sa True i False, tako da se radi zamena True u 1 i False u 0
train_test.replace({True: 1, False: 0}, inplace=True)

In [ ]:
# Odbacivanje kolone Survived
train_test = train_test.drop('Survived', axis=1)

In [ ]:
# Odvajanje svih redova kojima fali podatak u koloni Age
y = train_test[train_test['Age'].notna()]['Age']
# Odvajanje svih kategorija sa titulama, polom i godinama na osnovu kojih cemo dopuniti podatke koji fale u koloni Age
X = train_test[['Name_Capt.',
                'Name_Countess.',
                'Name_Don.',
                'Name_Dona.',
                'Name_Dr.',
                'Name_Lady.',
                'Name_Major.',
                'Name_Master.',
                'Name_Miss.',
                'Name_Mlle.',
                'Name_Mme.',
                'Name_Mr.',
                'Name_Mrs.',
                'Name_Rev.',
                'Name_Sir.',
                'Sex_female',
                'Sex_male',
                'Age']]

# Odbacivanje redova kojim fali podatak u koloni Age
X = X[X['Age'].notna()]
# Odbacivanje kolone Age
X = X.drop('Age', axis=1)

In [ ]:
# Odvajanje ostalih redova kojima fali podatak u koloni Age
X_missing = train_test[train_test['Age'].isnull()][['Name_Capt.',
                                                    'Name_Countess.',
                                                    'Name_Don.',
                                                    'Name_Dr.',
                                                    'Name_Lady.',
                                                    'Name_Major.',
                                                    'Name_Master.',
                                                    'Name_Miss.',
                                                    'Name_Mlle.',
                                                    'Name_Mme.',
                                                    'Name_Mr.',
                                                    'Name_Mrs.',
                                                    'Name_Rev.',
                                                    'Name_Sir.',
                                                    'Name_Dona.',
                                                    'Sex_female',
                                                    'Sex_male',]]

# Isprobao sam jos modela, ali KNeighborsRegressor daje najbolji rezultat
knn_regressor = KNeighborsRegressor(n_neighbors=17)
# Fit-ovanje modela X i y podacima
knn_regressor.fit(X, y)

In [ ]:
# Izdvajanje podataka nad kojim ce se vrsiti predvidjanje podataka koji fale u koloni Age
X_missing = X_missing[X.columns.to_list()]

In [ ]:
# Predvidjanje tih podataka
y_predicted = knn_regressor.predict(X_missing)
# Popunjavanje praznih polja u koloni Age sa predvidjenim podacima
train_test.loc[train_test['Age'].isnull(), 'Age'] = y_predicted

In [ ]:
plt.figure(figsize=(32,16))
sns.heatmap(train_test.corr(), annot=True, cmap='YlGnBu')

In [ ]:
columns_to_drop =  ['Name_Capt.',
                    'Name_Countess.',
                    'Name_Don.',
                    'Name_Dr.',
                    'Name_Lady.',
                    'Name_Major.',
                    'Name_Master.',
                    'Name_Miss.',
                    'Name_Mlle.',
                    'Name_Mme.',
                    'Name_Mr.',
                    'Name_Mrs.',
                    'Name_Rev.',
                    'Name_Sir.',
                    'Name_Dona.']

# Odbacivanje svih kolona sa titulama jer su nepotrebne za dalji proces
for column in columns_to_drop:
    train_test = train_test.drop(column, axis=1)

In [ ]:
# Popunjavanje jednog izgubljenog podatka iz kolone Fare srednjom vrednoscu preostalih podataka iste kolone
train_test.loc[train_test['PassengerId'] == 1044, 'Fare'] = train_test['Fare'].mean()

In [ ]:
plt.figure(figsize=(12,6))
sns.heatmap(train_test.corr(), annot=True, cmap='YlGnBu')

In [ ]:
train_test.hist(figsize=(14,10))

In [ ]:
# Skaliranje podataka ne doprinosi poboljsanju kod ovog problema, isti rezultat se dobija
#scaler = StandardScaler()
#scaled_data = scaler.fit_transform(train_test)
#scaled = pd.DataFrame(scaled_data, columns=train_test.columns)

In [ ]:
# Odvajanje training podataka
train = train_test.iloc[:891]
# Odvajanje test podataka
test = train_test.iloc[891:]

In [ ]:
model = LogisticRegression()

# Odvajanje y podataka za fit-ovanje modela 
survived = train_data['Survived']
# Odvajanje indeksa iz kolone PassengerId za koje se vrsi predvidjanje 
PassengerId = test['PassengerId']
# Odvajanje X podataka za fit-ovanje modela, ali se odbacuje kolona PassengerId zato sto nije od znacaja
train = train.drop('PassengerId', axis=1)
# Odvajanje X podataka sa kojima se vrsi predvidjanje, ali se odbacuje kolona PassengerId zato sto nije od znacaja
test = test.drop('PassengerId', axis=1)

In [ ]:
# Fit-ovanje modela train i survived podacima
model.fit(X=train, y=survived)

In [ ]:
# Predvidjanje modela test podacima
prediction = model.predict(X=test)

In [ ]:
submission = pd.DataFrame()
submission['PassengerId'] = PassengerId
submission['Survived'] = prediction

In [ ]:
# Ucitavanje stvarnih vrednosti koje je trebalo predvideti
real_data = pd.read_csv('./gender_submission.csv')

In [ ]:
# Eksportovanje tabele kao krajnji rezultat
submission.to_csv('submission.csv', index=False)